In [13]:
import numpy as np
import math
from cvxopt import solvers, matrix
from scipy.optimize import linprog

r = 0.05
backstop = 70
periods = 8
countries = 7
# inverse demand
odd_m = -0.4 # slope for odd numbered periods
odd_b = 199.78 # intercept for odd numbered periods
even_m = -0.37 # slope for even numbered periods
even_b = 214.15 # intercept for even numbered periods

# assume everyone maximally cheated in the first two rounds
reserves = [1080 - 240, 414 - 92, 333 - 74, 297 - 66, 270 - 60, 396 - 88, 243 - 54]
# everything divided by 100 to ease pressure on the optimizer
cap = [120, 46, 37, 33, 30, 44, 27]
mc = [9, 10, 16, 13, 5, 20, 7]

# variable layout: 10 quantities for KSA, 10 quantities for IRN, ... 10 prices]
variables = periods * (countries + 1)

In [14]:
# set up the optimization matrices
# objective function: social welfare
obj_P = np.zeros((variables, variables))
obj_q = np.zeros(variables)
# production capacity inequalities: upper bounds
G = np.zeros((variables + periods * countries + periods + countries, variables))
h = np.zeros(variables + periods * countries + periods + countries)
# inverse demand equalities
A = np.zeros((periods, variables))
b = np.zeros(periods)

In [15]:
def price_index (p):
    return periods * countries + p

def quantity_index (c, p):
    return periods * c + p

In [16]:
for p in range(periods):
    if p % 2 == 0:
        m = odd_m
        intercept = odd_b
    else:
        m = even_m
        intercept = even_b
    for c in range(countries):
        # extra term for (70 - MC_c) * (reserves_c - Q_c1 - Q_c2 - Q_c3...)
        
        # want to maximize (P_p - MC_c) * Q_cp * (1 + r)^(10 - p), made negative 
        discount_factor = math.pow(1 + r, periods - p)
        obj_P[quantity_index(c, p)][price_index(p)] = obj_P[price_index(p)][quantity_index(c, p)] = -1# * discount_factor
        obj_q[quantity_index(c, p)] = (discount_factor * mc[c] + 70 - mc[c]) / discount_factor
        
        G[variables + quantity_index(c, p)][quantity_index(c, p)] = 1
        h[variables + quantity_index(c, p)] = cap[c]
        
        G[quantity_index(c, p)][quantity_index(c, p)] = -1
        
        G[variables + periods * countries + p][quantity_index(c, p)] = -1 * m
        G[variables + periods * countries + periods + c][quantity_index(c, p)] = 1
        h[variables + periods * countries + periods + c] = reserves[c]
    G[price_index(p)][price_index(p)] = -1
    G[variables + periods * countries + p][price_index(p)] = 1
    h[variables + periods * countries + p] = intercept      
        

In [17]:
solvers.options['feastol']=1e-6
solvers.options['abstol']=1e-6
sol = solvers.qp(matrix(obj_P, tc='d'),matrix(obj_q, tc='d'),matrix(G, tc='d'),matrix(h, tc='d'))
print np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -4.1196e+05 -4.3028e+05  5e+06  2e+00  6e+00
 1: -1.3119e+05 -5.8813e+05  1e+06  3e-01  9e-01
 2: -1.1282e+05 -2.6280e+05  1e+05  1e-16  1e-15
 3: -1.1504e+05 -1.2042e+05  5e+03  1e-16  2e-15
 4: -1.1543e+05 -1.1695e+05  2e+03  1e-16  5e-14
 5: -1.1571e+05 -1.1632e+05  6e+02  1e-16  7e-14
 6: -1.1584e+05 -1.1607e+05  2e+02  1e-16  1e-13
 7: -1.1588e+05 -1.1598e+05  1e+02  1e-16  2e-13
 8: -1.1591e+05 -1.1593e+05  2e+01  1e-16  3e-13
 9: -1.1592e+05 -1.1592e+05  4e+00  2e-16  4e-13
10: -1.1592e+05 -1.1592e+05  5e-01  1e-16  6e-13
11: -1.1592e+05 -1.1592e+05  5e-02  2e-16  4e-13
Optimal solution found.
[[1.19996289e+02]
 [1.19996350e+02]
 [1.19993507e+02]
 [1.19301956e+02]
 [7.24801980e+01]
 [1.02736165e+02]
 [6.60496553e+01]
 [1.19443811e+02]
 [9.46593866e+00]
 [4.12591678e+01]
 [4.26575557e+00]
 [3.60478825e+01]
 [4.59953793e+01]
 [4.59959256e+01]
 [4.59976800e+01]
 [4.59977707e+01]
 [1.96021720e-04]
 [2.32956790e-04]
 [2.43406255e-04

In [18]:
# quantities
np.sum(np.array(sol['x'][:56]).reshape(7, -1), axis=0)

array([186.46206818, 218.25544059, 181.25897303, 212.34921879,
       175.4750201 , 205.73076176, 169.04508854, 198.43170694])

In [19]:
# prices
np.array(sol['x'])[56:]

array([[125.19517096],
       [133.39548547],
       [127.27640896],
       [135.5807875 ],
       [129.58999006],
       [138.02961655],
       [132.1619626 ],
       [140.73026677]])